In [1]:
from pmdarima import auto_arima
import pandas as pd

In [12]:
def get_stock_data(ticker, report_date):
    report_date = pd.to_datetime(report_date)
    end_date = report_date + pd.DateOffset(days=10)
    start_date = report_date - pd.DateOffset(years=1)

    end_date = end_date.strftime('%Y-%m-%d')
    start_date = start_date.strftime('%Y-%m-%d')

    filename = f"../dataset/close-prices-for-bankrupt-companies/{ticker}_closeprice_gurufocus.csv"
    
    stock_close_prices = pd.read_csv(filename)
    
    stock_data = stock_close_prices[(stock_close_prices['date'] >= start_date) & (stock_close_prices['date'] <= end_date)]

    return stock_data

In [10]:
from data_utils import get_positive_samples_ticker_and_report_dates
positive_samples = get_positive_samples_ticker_and_report_dates()
positive_samples

,ticker,subset,report_datetime
0,GOTTQ,test,2008-04-10
1,SGICQ,train,2008-09-22
2,BRLCQ,train,2007-09-13
3,TMYEQ,train,2008-03-31
4,SHRPQ,train,2007-05-31
...,...,...,...
69,DNDNQ,train,2014-03-03
70,FNLYQ,train,2009-05-01
71,MRIIQ,test,2000-03-30
72,FFNTQ,test,2013-04-01


In [13]:
historical_stock_prices = get_stock_data(positive_samples['ticker'][0], positive_samples['report_datetime'][0])
historical_stock_prices

Reading ../dataset/close-prices-for-bankrupt-companies/GOTTQ_closeprice_gurufocus.csv from 2007-04-10 to 2008-04-20, ticker: GOTTQ
5000


,date,close_price
4315,2007-04-10,14.80
4316,2007-04-11,14.56
4317,2007-04-12,13.92
4318,2007-04-13,13.97
4319,2007-04-16,14.25
...,...,...
4570,2008-04-14,2.57
4571,2008-04-15,2.38
4572,2008-04-16,2.11
4573,2008-04-17,2.18


In [22]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

historical_stock_prices.plot()
result = sm.tsa.stattools.adfuller(historical_stock_prices['close_price'])
plot_acf(historical_stock_prices['close_price'])
plot_pacf(historical_stock_prices['close_price'])

In [26]:
model = auto_arima(historical_stock_prices['close_price'], 
                    start_p=1, start_q=1, 
                    max_p=5, max_q=5, 
                    d=1,
                    seasonal=False, 
                    stepwise=True,
                    trace=True,   
                    suppress_warnings=True,
                    maxiter=20,
                    information_criterion='aic')
print(model.summary())

Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=23.054, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=24.735, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=21.083, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=21.532, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=32.061, Time=0.00 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=23.005, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=23.953, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=25.823, Time=0.02 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.283 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                  260
Model:               SARIMAX(1, 1, 0)   Log Likelihood                  -7.542
Date:                Sun, 22 Sep 2024   AIC                             21.083
Time:                        11:03:42   BIC         

In [25]:
model.fit(historical_stock_prices['close_price'])

future_forecast = model.predict(n_periods=10)
print(future_forecast)

forecast, conf_int = model.predict(n_periods=10, return_conf_int=True)
forecast

260    2.287952
261    2.243563
262    2.195890
263    2.147735
264    2.099509
265    2.051272
266    2.003034
267    1.954796
268    1.906558
269    1.858320
dtype: float64


C:\Users\grzeg\PycharmProjects\bankruptcy-prophet-new\bankruptcy-prophet\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\grzeg\PycharmProjects\bankruptcy-prophet-new\bankruptcy-prophet\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\grzeg\PycharmProjects\bankruptcy-prophet-new\bankruptcy-prophet\.venv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\grzeg\PycharmProjects\bankruptcy-prophet-new\bankruptcy-prophet\.venv\Lib\site-packages

260    2.287952
261    2.243563
262    2.195890
263    2.147735
264    2.099509
265    2.051272
266    2.003034
267    1.954796
268    1.906558
269    1.858320
dtype: float64